# Building Recommendation Engine Using Factorization Machines

## Prepare data set using MovieLens

This lab is an extended version of AWS's ML blog [Build a movie recommender with factorization machines on Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/build-a-movie-recommender-with-factorization-machines-on-amazon-sagemaker/). In the origianl lab available at [GitHub](https://github.com/juliensimon/dlnotebooks/blob/master/sagemaker/03-Factorization-Machines-Movielens.ipynb), it used 100k movie rating data that achieved around 71.5% accuracy.

In this lab, we will utilize the same public benchmarking data set known as [MovieLens](https://grouplens.org/datasets/movielens/) but much bigger than the previous one. The data we will use is composed of 20 million ratings and 465,000 tag applications applied to 27,000 movies by 138,000 users. Includes tag genome data with 12 million relevance scores across 1,100 tags.

The data structure is a little different with 100k samples. So the below code flow would be a bit different with [the original version]((https://github.com/juliensimon/dlnotebooks/blob/master/sagemaker/03-Factorization-Machines-Movielens.ipynb).

Let's see if the bigger data set contributes to the predction accuracy.

## Load python modules and define working environment

In [1]:
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import json_deserializer

import boto3, csv, io, json
import numpy as np
from scipy.sparse import lil_matrix

In [2]:
bucket = 'pilho-sagemaker-ai-workshop'
prefix = 'sagemaker/fm-movielens'

## Download data

In [1]:
!wget -P /tmp/movielens/ http://files.grouplens.org/datasets/movielens/ml-20m.zip

--2018-08-29 04:55:47--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘/tmp/movielens/ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  56.8MB/s    in 3.5s    

2018-08-29 04:55:51 (54.4 MB/s) - ‘/tmp/movielens/ml-20m.zip’ saved [198702078/198702078]



In [2]:
!unzip /tmp/movielens/ml-20m.zip -d/tmp/movielens/data

Archive:  /tmp/movielens/ml-20m.zip
   creating: /tmp/movielens/data/ml-20m/
  inflating: /tmp/movielens/data/ml-20m/genome-scores.csv  
  inflating: /tmp/movielens/data/ml-20m/genome-tags.csv  
  inflating: /tmp/movielens/data/ml-20m/links.csv  
  inflating: /tmp/movielens/data/ml-20m/movies.csv  
  inflating: /tmp/movielens/data/ml-20m/ratings.csv  
  inflating: /tmp/movielens/data/ml-20m/README.txt  
  inflating: /tmp/movielens/data/ml-20m/tags.csv  


In [10]:
!head -10 /tmp/movielens/data/ml-20m/genome-scores.csv

In [13]:
!head -10 /tmp/movielens/data/ml-20m/tags.csv 

In [12]:
!head -10 /tmp/movielens/data/ml-20m/links.csv

In [3]:
!head -10 /tmp/movielens/data/ml-20m/ratings.csv 

In [4]:
!head -10 /tmp/movielens/data/ml-20m/movies.csv 

Find the total number of movies

In [8]:
!tail -10 /tmp/movielens/data/ml-20m/movies.csv 

In [9]:
!wc -l /tmp/movielens/data/ml-20m/movies.csv 

27279 /tmp/movielens/data/ml-20m/movies.csv


In [ ]:
movieCount = 27279

In [7]:
!tail -10 /tmp/movielens/data/ml-20m/ratings.csv 

In [ ]:
userCount = 138493

In [14]:
!wc -l /tmp/movielens/data/ml-20m/ratings.csv 

20000264 /tmp/movielens/data/ml-20m/ratings.csv


In [5]:
!cat /tmp/movielens/data/ml-20m/README.txt 

Summary

This dataset (ml-20m) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 20000263 ratings and 465564 tag applications across 27278 movies. These data were created by 138493 users between January 09, 1995 and March 31, 2015. This dataset was generated on March 31, 2015, and updated on October 17, 2016 to update links.csv and add genome-* files.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in six files, `genome-scores.csv`, `genome-tags.csv`, `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This and other GroupLens data sets are publicly available for download at <http://grouplens.org/datasets/>.


Usage License

Neither the Universi

Let's shuffle input data. We remove the first line which includes column headers.

In [2]:
%cd /tmp/movielens/ml-20m
!tail -n +2 ratings.csv | shuf -o ratings.shuffled.csv

/home/ec2-user/SageMaker/pilho-lab/ml-100k
343	25	2	876402814
659	212	4	891387227
858	515	4	880932911
387	196	2	886484012
854	762	2	882812905
421	466	4	892241459
436	895	4	887768717
605	462	5	881016176
346	276	1	874950029
40	294	4	889041671


In [15]:
!head -10 /tmp/movielens/data/ml-20m/ratings.shuffled.csv

Let's separate the training set and the test set

In [5]:
trainingDataRatio = 0.7
totalSampleLength = 20000263
trainingDataLength = int(totalSampleLength * trainingDataRatio)
testingDataLength = totalSampleLength - trainingDataLength

In [6]:
print(trainingDataLength, testingDataLength)

(14000184, 6000079)


In [22]:
!head -14000184 /tmp/movielens/data/ml-20m/ratings.shuffled.csv > /tmp/movielens/data/ml-20m/ratings.train.shuffled.csv

In [23]:
!head -6000079 /tmp/movielens/data/ml-20m/ratings.shuffled.csv > /tmp/movielens/data/ml-20m/ratings.test.shuffled.csv

### Build training set and test set

In [7]:
nbUsers=138493
nbMovies=131262
nbFeatures=nbUsers+nbMovies

nbRatingsTrain=trainingDataLength
nbRatingsTest=testingDataLength

In [8]:
traingDataSet = '/tmp/movielens/data/ml-20m/ratings.train.shuffled.csv'
testingDataSet = '/tmp/movielens/data/ml-20m/ratings.test.shuffled.csv'

In [28]:
# For each user, build a list of rated movies.
# We'd need this to add random negative samples.
moviesByUser = {}
for userId in range(nbUsers):
    moviesByUser[str(userId)]=[]
 
with open(traingDataSet,'r') as f:
    samples=csv.reader(f,delimiter=',')
    for userId,movieId,rating,timestamp in samples:
        moviesByUser[str(int(userId)-1)].append(int(movieId)-1) 

In [36]:
def loadDataset(filename, lines, columns):
    # Features are one-hot encoded in a sparse matrix
    X = lil_matrix((lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line=0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter=',')
        for userId,movieId,rating,timestamp in samples:
            X[line,int(userId)-1] = 1
            X[line,int(nbUsers)+int(movieId)-1] = 1
            # Note that this big rating data has a float review score
            if float(rating) >= 4.0:
                Y.append(1)
            else:
                Y.append(0)
            line=line+1
            
    Y=np.array(Y).astype('float32')
    return X,Y

In [34]:
X_train, Y_train = loadDataset(traingDataSet, nbRatingsTrain, nbFeatures)
X_test, Y_test = loadDataset(testingDataSet,nbRatingsTest,nbFeatures)

In [37]:
print(X_train[1000])

  (0, 44951)	1.0
  (0, 172828)	1.0


In [38]:
print(X_train.shape)
print(Y_train.shape)
assert X_train.shape == (nbRatingsTrain, nbFeatures)
assert Y_train.shape == (nbRatingsTrain, )
zero_labels = np.count_nonzero(Y_train)
print("Training labels: %d zeros, %d ones" % (zero_labels, nbRatingsTrain-zero_labels))

print(X_test.shape)
print(Y_test.shape)
assert X_test.shape  == (nbRatingsTest, nbFeatures)
assert Y_test.shape  == (nbRatingsTest, )
zero_labels = np.count_nonzero(Y_test)
print("Test labels: %d zeros, %d ones" % (zero_labels, nbRatingsTest-zero_labels))

(14000184, 269755)
(14000184,)
Training labels: 6996026 zeros, 7004158 ones
(6000079, 269755)
(6000079,)
Test labels: 2998296 zeros, 3001783 ones


### Convert to protobuf and save to S3

In [9]:
train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train3')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test3')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

In [10]:
def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, X, Y)
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)

In [ ]:
train_data = writeDatasetToProtobuf(X_train, Y_train, bucket, train_prefix, train_key)    
test_data  = writeDatasetToProtobuf(X_test, Y_test, bucket, test_prefix, test_key)    
  
print(train_data)
print(test_data)
print('Output: {}'.format(output_prefix))

### Run training job

In [11]:
train_data = 's3://pilho-sagemaker-ai-workshop/sagemaker/fm-movielens/train3/train.protobuf'
test_data = 's3://pilho-sagemaker-ai-workshop/sagemaker/fm-movielens/test3/test.protobuf'
output_prefix = 's3://pilho-sagemaker-ai-workshop/sagemaker/fm-movielens/output'

In [12]:
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'factorization-machines')

In [ ]:
fm = sagemaker.estimator.Estimator(container,
                                   get_execution_role(), 
                                   train_instance_count=4, 
                                   train_instance_type='ml.c5.18xlarge',
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())

fm.set_hyperparameters(feature_dim=nbFeatures,
                      predictor_type='binary_classifier',
                      mini_batch_size=1000,
                      num_factors=64,
                      epochs=100)

fm.fit({'train': train_data, 'test': test_data})

INFO:sagemaker:Creating training-job with name: factorization-machines-2018-08-29-13-05-40-942


.....................
Docker entrypoint called with argument(s): train
Docker entrypoint called with argument(s): train
[08/29/2018 13:09:06 INFO 140667034941248] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'factors_init_sigma': u'0.001', u'_log_level': u'info', u'bias_init_method': u'normal', u'linear_init_method': u'normal', u'linear_lr': u'0.001', u'factors_init_method': u'normal', u'_tuning_objective_metric': u'', u'bias_wd': u'0.01', u'use_linear': u'true', u'bias_lr': u'0.1', u'mini_batch_size': u'1000', u'_use_full_symbolic': u'true', u'batch_metrics_publish_interval': u'500', u'bias_init_sigma': u'0.01', u'_num_gpus': u'auto', u'_data_format': u'record', u'factors_wd': u'0.00001', u'linear_wd': u'0.001', u'_kvstore': u'auto', u'_learning_rate': u'1.0', u'_optimizer': u'ad

[08/29/2018 13:09:35 INFO 140658308478784] Iter[0] Batch [500]#011Speed: 23141.85 samples/sec
[08/29/2018 13:09:35 INFO 140658308478784] #quality_metric: host=algo-2, epoch=0, batch=500 train binary_classification_accuracy <score>=0.572219560878
[08/29/2018 13:09:35 INFO 140658308478784] #quality_metric: host=algo-2, epoch=0, batch=500 train binary_classification_cross_entropy <loss>=0.6760168228
[08/29/2018 13:09:35 INFO 140658308478784] #quality_metric: host=algo-2, epoch=0, batch=500 train binary_f_1.000 <score>=0.572067520995
[08/29/2018 13:09:35 INFO 140667034941248] Iter[0] Batch [500]#011Speed: 23374.13 samples/sec
[08/29/2018 13:09:35 INFO 140667034941248] #quality_metric: host=algo-3, epoch=0, batch=500 train binary_classification_accuracy <score>=0.574399201597
[08/29/2018 13:09:35 INFO 140667034941248] #quality_metric: host=algo-3, epoch=0, batch=500 train binary_classification_cross_entropy <loss>=0.674983392224
[08/29/2018 13:09:35 INFO 140667034941248] #quality_metric: ho

[08/29/2018 13:11:04 INFO 140658308478784] Iter[0] Batch [2500]#011Speed: 21393.92 samples/sec
[08/29/2018 13:11:04 INFO 140658308478784] #quality_metric: host=algo-2, epoch=0, batch=2500 train binary_classification_accuracy <score>=0.623036385446
[08/29/2018 13:11:04 INFO 140658308478784] #quality_metric: host=algo-2, epoch=0, batch=2500 train binary_classification_cross_entropy <loss>=0.656341308447
[08/29/2018 13:11:04 INFO 140658308478784] #quality_metric: host=algo-2, epoch=0, batch=2500 train binary_f_1.000 <score>=0.621729302131
[08/29/2018 13:11:05 INFO 140667034941248] Iter[0] Batch [2500]#011Speed: 21252.25 samples/sec
[08/29/2018 13:11:05 INFO 140667034941248] #quality_metric: host=algo-3, epoch=0, batch=2500 train binary_classification_accuracy <score>=0.623303878449
[08/29/2018 13:11:05 INFO 140667034941248] #quality_metric: host=algo-3, epoch=0, batch=2500 train binary_classification_cross_entropy <loss>=0.656142100752
[08/29/2018 13:11:05 INFO 140667034941248] #quality_m

[08/29/2018 13:12:38 INFO 140658308478784] Iter[0] Batch [4500]#011Speed: 21084.36 samples/sec
[08/29/2018 13:12:38 INFO 140658308478784] #quality_metric: host=algo-2, epoch=0, batch=4500 train binary_classification_accuracy <score>=0.64327438347
[08/29/2018 13:12:38 INFO 140658308478784] #quality_metric: host=algo-2, epoch=0, batch=4500 train binary_classification_cross_entropy <loss>=0.646116332682
[08/29/2018 13:12:38 INFO 140658308478784] #quality_metric: host=algo-2, epoch=0, batch=4500 train binary_f_1.000 <score>=0.642830004004
[08/29/2018 13:12:40 INFO 140667034941248] Iter[0] Batch [4500]#011Speed: 20414.06 samples/sec
[08/29/2018 13:12:40 INFO 140667034941248] #quality_metric: host=algo-3, epoch=0, batch=4500 train binary_classification_accuracy <score>=0.644221950678
[08/29/2018 13:12:40 INFO 140667034941248] #quality_metric: host=algo-3, epoch=0, batch=4500 train binary_classification_cross_entropy <loss>=0.64544417663
[08/29/2018 13:12:40 INFO 140667034941248] #quality_met

...

[08/29/2018 13:42:49 INFO 140009951377216] Iter[3] Batch [1000]#011Speed: 21607.65 samples/sec
[08/29/2018 13:42:49 INFO 140009951377216] #quality_metric: host=algo-1, epoch=3, batch=1000 train binary_classification_accuracy <score>=0.729478521479
[08/29/2018 13:42:49 INFO 140009951377216] #quality_metric: host=algo-1, epoch=3, batch=1000 train binary_classification_cross_entropy <loss>=0.540287957678
[08/29/2018 13:42:49 INFO 140009951377216] #quality_metric: host=algo-1, epoch=3, batch=1000 train binary_f_1.000 <score>=0.734409848703
[08/29/2018 13:42:54 INFO 140667034941248] Iter[3] Batch [500]#011Speed: 22960.79 samples/sec
[08/29/2018 13:42:54 INFO 140667034941248] #quality_metric: host=algo-3, epoch=3, batch=500 train binary_classification_accuracy <score>=0.733157684631
[08/29/2018 13:42:54 INFO 140667034941248] #quality_metric: host=algo-3, epoch=3, batch=500 train binary_classification_cross_entropy <loss>=0.536878102011
[08/29/2018 13:42:54 INFO 140667034941248] #quality_metr

[08/29/2018 13:44:46 INFO 140658308478784] Iter[3] Batch [3000]#011Speed: 21893.27 samples/sec
[08/29/2018 13:44:46 INFO 140658308478784] #quality_metric: host=algo-2, epoch=3, batch=3000 train binary_classification_accuracy <score>=0.731613462179
[08/29/2018 13:44:46 INFO 140658308478784] #quality_metric: host=algo-2, epoch=3, batch=3000 train binary_classification_cross_entropy <loss>=0.537194429972
[08/29/2018 13:44:46 INFO 140658308478784] #quality_metric: host=algo-2, epoch=3, batch=3000 train binary_f_1.000 <score>=0.736197006232
[08/29/2018 13:44:50 INFO 140009951377216] Iter[3] Batch [3500]#011Speed: 22471.47 samples/sec
[08/29/2018 13:44:50 INFO 140009951377216] #quality_metric: host=algo-1, epoch=3, batch=3500 train binary_classification_accuracy <score>=0.730838046272
[08/29/2018 13:44:50 INFO 140009951377216] #quality_metric: host=algo-1, epoch=3, batch=3500 train binary_classification_cross_entropy <loss>=0.538389810614
[08/29/2018 13:44:50 INFO 140009951377216] #quality_m

[08/29/2018 13:46:09 INFO 140667034941248] Iter[3] Batch [4500]#011Speed: 19838.96 samples/sec
[08/29/2018 13:46:09 INFO 140667034941248] #quality_metric: host=algo-3, epoch=3, batch=4500 train binary_classification_accuracy <score>=0.733822706065
[08/29/2018 13:46:09 INFO 140667034941248] #quality_metric: host=algo-3, epoch=3, batch=4500 train binary_classification_cross_entropy <loss>=0.534671493534
[08/29/2018 13:46:09 INFO 140667034941248] #quality_metric: host=algo-3, epoch=3, batch=4500 train binary_f_1.000 <score>=0.738614065303
[08/29/2018 13:46:22 INFO 140658308478784] Iter[3] Batch [5000]#011Speed: 20267.91 samples/sec
[08/29/2018 13:46:22 INFO 140658308478784] #quality_metric: host=algo-2, epoch=3, batch=5000 train binary_classification_accuracy <score>=0.732214157169
[08/29/2018 13:46:22 INFO 140658308478784] #quality_metric: host=algo-2, epoch=3, batch=5000 train binary_classification_cross_entropy <loss>=0.536438374209
[08/29/2018 13:46:22 INFO 140658308478784] #quality_m

[08/29/2018 13:48:01 INFO 140658308478784] Iter[3] Batch [7000]#011Speed: 18506.97 samples/sec
[08/29/2018 13:48:01 INFO 140658308478784] #quality_metric: host=algo-2, epoch=3, batch=7000 train binary_classification_accuracy <score>=0.732130267105
[08/29/2018 13:48:01 INFO 140658308478784] #quality_metric: host=algo-2, epoch=3, batch=7000 train binary_classification_cross_entropy <loss>=0.53621455598
[08/29/2018 13:48:01 INFO 140658308478784] #quality_metric: host=algo-2, epoch=3, batch=7000 train binary_f_1.000 <score>=0.737054384778
[08/29/2018 13:48:04 INFO 140009951377216] Iter[3] Batch [7500]#011Speed: 19724.35 samples/sec
[08/29/2018 13:48:04 INFO 140009951377216] #quality_metric: host=algo-1, epoch=3, batch=7500 train binary_classification_accuracy <score>=0.73111105186
[08/29/2018 13:48:04 INFO 140009951377216] #quality_metric: host=algo-1, epoch=3, batch=7500 train binary_classification_cross_entropy <loss>=0.537486771564
[08/29/2018 13:48:04 INFO 140009951377216] #quality_met

[08/29/2018 13:49:34 INFO 140009951377216] Iter[3] Batch [9500]#011Speed: 21190.56 samples/sec
[08/29/2018 13:49:34 INFO 140009951377216] #quality_metric: host=algo-1, epoch=3, batch=9500 train binary_classification_accuracy <score>=0.730824965793
[08/29/2018 13:49:34 INFO 140009951377216] #quality_metric: host=algo-1, epoch=3, batch=9500 train binary_classification_cross_entropy <loss>=0.537443075164
[08/29/2018 13:49:34 INFO 140009951377216] #quality_metric: host=algo-1, epoch=3, batch=9500 train binary_f_1.000 <score>=0.73589027945
[08/29/2018 13:49:35 INFO 140658308478784] Iter[3] Batch [9000]#011Speed: 21690.33 samples/sec
[08/29/2018 13:49:35 INFO 140658308478784] #quality_metric: host=algo-2, epoch=3, batch=9000 train binary_classification_accuracy <score>=0.731989001222
[08/29/2018 13:49:35 INFO 140658308478784] #quality_metric: host=algo-2, epoch=3, batch=9000 train binary_classification_cross_entropy <loss>=0.536130807328
[08/29/2018 13:49:35 INFO 140658308478784] #quality_me

### Deploy model

In [ ]:
fm_predictor = fm.deploy(instance_type='ml.c4.xlarge', initial_instance_count=1)

In [ ]:
def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    #print js
    return json.dumps(js)

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

### Run predictions

In [ ]:
print(X_test[1000:1010])

In [ ]:
result = fm_predictor.predict(X_test[2000:2010].toarray())
print(result)

In [ ]:
print (Y_test[2000:2010])

In [ ]:
print(X_test[2000])

In [ ]:
result = fm_predictor.predict(X_test[2000:2001].toarray())
print(result)
print(Y_test[2000:2001])